# A/B Test Review

In [213]:
import pandas as pd

## Load log file

In [214]:
df = pd.read_json('../website/logs/app.log', lines=True)

In [215]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [216]:
df.head(2)

,level,message,timestamp
0,info,"{'event': 'impression', 'experiments': [{'grou...",2024-04-08 06:36:20.314000+00:00
1,info,"{'event': 'click', 'experiments': [], 'extra':...",2024-04-08 06:36:21.918000+00:00


In [217]:
message_df = pd.json_normalize(df['message'])
message_df.head(2)

,event,experiments,page,url,username,extra.action,extra.elementId,extra.page,extra.productId,extra.paymentId
0,impression,"[{'group': 'test', 'name': 'SkipConfirmationSc...",,/,53e71b03-794d-4de9-8ae3-376980ee4ffb,NaN,NaN,NaN,NaN,NaN
1,click,[],welcome,/log,27b63d40-3661-4cf4-b531-0fc5a169b4e4,click,startButton,welcome,NaN,NaN


In [218]:
experiments_df = pd.json_normalize(df['message'], record_path='experiments')
experiments_df.head(2)

,group,name
0,test,SkipConfirmationScreen
1,test,SkipConfirmationScreen


In [219]:
df = pd.concat([df, message_df, experiments_df], axis=1).drop([
    'message', 
    'experiments', 
    'url',
    'level',
    'extra.elementId',
    'extra.productId',
    'extra.paymentId',
    'extra.action',
    'extra.page',
], axis=1)

In [220]:
df = df.sort_values(['username', 'timestamp'])

In [212]:
df.head(2)

,level,message,timestamp
0,info,"{'event': 'impression', 'experiments': [{'grou...",2024-04-08 06:36:20.314000+00:00
1,info,"{'event': 'click', 'experiments': [], 'extra':...",2024-04-08 06:36:21.918000+00:00


## Are test and control groups balanced?

In [191]:
df.groupby(['name', 'group'])['username'].nunique().reset_index(name='unique_count')

,name,group,unique_count
0,SkipConfirmationScreen,test,2


## Time spent

In [231]:
df['time_diff'] = df.groupby(['username', 'group'])['timestamp'].diff().fillna(pd.Timedelta(seconds=0))
print(
    'Time spent:',
    df[df['group'] == 'test']['time_diff'].mean(), '(test)',
    df[df['group'] == 'control']['time_diff'].mean() , '(control)'
)

Time spent: 0 days 00:00:00.271166666 (test) NaT (control)
